In [34]:
import pandas as pd
import nltk
april_hh = pd.read_excel(r'C:\Users\n1555085\Downloads\Copy of CSAT Help hub April Responses.xlsx')


In [47]:
courtesy = april_hh['Unnamed: 9'].dropna().values.tolist()[1:]
# convert uppercase to lower
effectiveness = april_hh['Unnamed: 10'].dropna().values.tolist()[1:]
timeliness = april_hh['Unnamed: 11'].dropna().values.tolist()[1:]
understanding = april_hh['Unnamed: 12'].dropna().values.tolist()[1:]
NPS = april_hh['Unnamed: 13'].dropna().values.tolist()[1:]
comments = courtesy + effectiveness + timeliness + understanding + NPS

print(f'# of courtesy comments: {len(courtesy)}')
print(f'# of effectiveness comments: {len(effectiveness)}')
print(f'# of timeliness comments: {len(timeliness)}')
print(f'# of understanding comments: {len(understanding)}')
print(f'# of nps comments: {len(NPS)}')
print(f'# of comments: {len(comments)}')

# of courtesy comments: 16
# of effectiveness comments: 29
# of timeliness comments: 35
# of understanding comments: 29
# of nps comments: 169
# of comments: 278


In [48]:
#Initial comment processing

In [92]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from string import punctuation
customSW = set(stopwords.words('english') + list(punctuation))
def removeStopWords(sentences):
    toReturn = [None] * len(sentences)
    for i in range(len(sentences)):
        withoutStopwords = [word for word in sentences[i].lower().split() if word not in customSW]
        toReturn[i] = ' '.join(withoutStopwords)
    return toReturn
comments = removeStopWords(comments)



#customSW
#comments

In [93]:
#CLUSTERING

In [94]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np
from collections import defaultdict
from nltk.probability import FreqDist
from heapq import nlargest


In [95]:
vectorizer = TfidfVectorizer(max_df = 0.5, min_df = 3)
X = vectorizer.fit_transform(comments)
X
print(comments[0])
print(X[0])

already cleared cookies cache site still accessible. presumptuous answer.
  (0, 14)	0.519793378442015
  (0, 207)	0.4060783032559657
  (0, 198)	0.5428905037670378
  (0, 10)	0.519793378442015


In [96]:
km = KMeans(n_clusters = 5, init = 'k-means++', max_iter = 100, n_init = 1, verbose = True)
km.fit(X)

Initialization complete
Iteration 0, inertia 261.55197729780446
Iteration 1, inertia 249.10940527103858
Iteration 2, inertia 247.50397327502418
Iteration 3, inertia 246.67296581245463
Iteration 4, inertia 246.38034421433557
Iteration 5, inertia 246.3257198548414
Converged at iteration 5: strict convergence.


KMeans(max_iter=100, n_clusters=5, n_init=1, verbose=True)

In [97]:
np.unique(km.labels_, return_counts = True)
text = {}
for i, cluster in enumerate(km.labels_):
    comment = comments[i]
    if cluster not in text.keys():
        text[cluster] = comment
    else: 
        text[cluster] += comment
keywords = {}
counts = {}
for cluster in range(5):
    word_sent = word_tokenize(text[cluster].lower())
    frq = FreqDist(word_sent)
    keywords[cluster] = nlargest(10, frq, key=frq.get)
    counts[cluster] = frq
keywords

{0: ['resolved',
  'ticket',
  'say',
  '.',
  'emailed',
  'made',
  'updates',
  'responded',
  'wasnt',
  'received'],
 1: ['fixed',
  ',',
  'problem',
  'resolution',
  '.',
  'nothing',
  'ongoing',
  'courteous',
  '!',
  'issue'],
 2: ['.',
  ',',
  'issue',
  'ticket',
  'resolved',
  'problem',
  'time',
  'help',
  'get',
  '!'],
 3: ['issue',
  'work',
  'resolve',
  'rather',
  'ended',
  'providing',
  'around.built',
  'around',
  'addressed'],
 4: ['professional',
  ',',
  '.',
  'courteous',
  'support',
  'representatives',
  'times.personnel',
  'professionaltimely',
  'professionalrichard',
  'tedesco']}